In [2]:
import numpy as np
import random
import tensorflow as tf
from collections import deque
import gym
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [3]:
env_name = 'CartPole-v1'
env = gym.make(env_name, render_mode='human')


In [4]:
env.reset()
for step in range(100):
    env.render()
    env.step(env.action_space.sample())

env.close()

C:\Users\bherw\OneDrive\Desktop\Suraj\Reinforment Learning\gridworld\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
C:\Users\bherw\OneDrive\Desktop\Suraj\Reinforment Learning\gridworld\venv\Lib\site-packages\gym\envs\classic_control\cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [5]:
num_of_obs = env.observation_space.shape[0]
num_of_act = env.action_space.n

In [6]:
model = Sequential()
model.add(Dense(16, activation = 'relu', input_shape=(1, num_of_obs)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(num_of_act, activation = 'linear'))



In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 dense_1 (Dense)             (None, 1, 32)             544       
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
Total params: 690 (2.70 KB)
Trainable params: 690 (2.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
target_model = clone_model(model)

In [9]:
EPOCH = 10
epsilon = 1.0
epsilon_reduce = 0.0001
learning_rate = 0.001
gamma = 0.95

In [10]:
def egas(model, epsilon, obs):
    if np.random.random() > epsilon:
        prediction = model.predict(obs)
        action = np.argmax(prediction)

    else:
        action = env.action_space.sample()

    return action

In [11]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [ ]:
def replay(replay_buffer, batch_size, model, target_model):
    if len(replay_buffer) < batch_size:
        return
    samples = random.sample(replay_buffer, batch_size)
    zipped_samples = list(zip(*samples))
    states, action, rewards, new_states, dones = zipped_samples
    targets = target_model.predict(np.array(states))
    q_value = model.predict(np.array(new_states))

